In [1]:
import argparse
from math import log10
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.cuda as cuda

import time
import numpy as np
import os
from RANet_lib import *
from RANet_lib.RANet_lib import *
from RANet_model import RANet as Net
import os
import os.path as osp
from glob import glob

import matplotlib.pyplot as plt
from torchvision import transforms
import PIL.Image as Image

In [2]:
# params='RANet_video_multi.pth'
params='RANet_multi_retrained1_epoch1.pth'
dataset='17val'
save_root='../predictions/RANet_Video_17val'

net_name = 'RANet'
parser = argparse.ArgumentParser(description='RANet')
parser.add_argument('--deviceID', default=[0], help='device IDs')
parser.add_argument('--threads', type=int, default=16, help='number of threads for data loader to use')
parser.add_argument('--workfolder', default='../models/')
parser.add_argument('--savePName', default=net_name)
parser.add_argument('--net_type', default='single_object')
parser.add_argument('--fp16', default=True)
print('===> Setting ......')
# opt = parser.parse_args()
class args:
    pass
opt = args()
opt.deviceID = [0]
opt.threads=0
opt.workfolder='../models/'
opt.savePName = net_name
opt.net_type='single_object'
opt.fp16=False

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

try:
    os.mkdir(opt.workfolder)
    print('build working folder: ' + opt.workfolder)
except:
    print(opt.workfolder + 'exists')
print('using device ID: {}'.format(opt.deviceID))


print('===> Building model')
params='RANet_video_multi.pth'
dataset='17train'
save_root='../predictions/RANet_Video_17val'

inSize1 = 480
inSize2 = 864
root = '../datasets/DAVIS'
img_mode = '480p'
img_shape = (inSize1,inSize2)

year = '2017'
DAVIS = dict(reading_type='SVOS',
                 year=year,
             root='../datasets/DAVIS/',
             subfolder=['', '', ''],
             mode=dataset,
             tar_mode='rep',
             train=0, val=0, test=0, predict=1,
             length=None,
             init_folder=None,
             )
dataset = DAVIS2017_loader(
    [DAVIS], mode='test',
    transform=[PAD_transform([inSize1, inSize2], random=False),
               PAD_transform([inSize1, inSize2], random=False)],
    rand=Rand_num())
data_loader_RAnet = DataLoader(dataset=dataset, num_workers=0, batch_size=1, shuffle=False, pin_memory=True)

===> Setting ......
../models/exists
using device ID: [0]
===> Building model
Number of files: 60
Dirs: 60
saving data in X_test


In [26]:
from vj_davis_17_loader import Custom_DAVIS2017_dataset
from torch.utils.data import DataLoader
from vj_loss_functions import *
img_dataset = Custom_DAVIS2017_dataset(root=root, img_shape=img_shape, img_mode=img_mode,loader_type='train')

trnsfm = transforms.ToPILImage()

loading files from:  ../datasets/DAVIS/ImageSets/2017/train.txt


In [37]:
img_dataset = Custom_DAVIS2017_dataset(root=root, img_shape=img_shape, img_mode=img_mode,loader_type='train')
for iteration, batch in enumerate(data_loader_RAnet, 1):
    name = img_dataset.names[iteration-1]
    print("Image loader counter for:",iteration-1,"name:", name, ": ", img_dataset.img_counter[name])
    for it in range(1,len(batch[0])):
        template,template_mask, target,target_mask = img_dataset.__getitem__(iteration-1)
        diff = (target-batch[0][it][0]).abs().sum() + (template-batch[0][it-1][0]).abs().sum()
        if (diff.item() > 0.0001):
            print("Difference in images!!!!")
            asdsad
        diff += (target_mask-batch[1][it][0]).abs().sum() + (template_mask-batch[1][it-1][0]).abs().sum()
        if (diff.item() > 0.0001):
            print("Difference in masks!!!!")
            asdsad
    print("Image loader Final counter for:",iteration,"name:", name, ": ", img_dataset.img_counter[name])

loading files from:  ../datasets/DAVIS/ImageSets/2017/train.txt
Image loader counter for: 0 name: bear :  1
Image loader Final counter for: 1 name: bear :  1
Image loader counter for: 1 name: bmx-bumps :  1
Image loader Final counter for: 2 name: bmx-bumps :  1
Image loader counter for: 2 name: boat :  1
Image loader Final counter for: 3 name: boat :  1
Image loader counter for: 3 name: boxing-fisheye :  1
Image loader Final counter for: 4 name: boxing-fisheye :  1
Image loader counter for: 4 name: breakdance-flare :  1
Image loader Final counter for: 5 name: breakdance-flare :  1
Image loader counter for: 5 name: bus :  1
Image loader Final counter for: 6 name: bus :  1
Image loader counter for: 6 name: car-turn :  1
Image loader Final counter for: 7 name: car-turn :  1
Image loader counter for: 7 name: cat-girl :  1
Image loader Final counter for: 8 name: cat-girl :  1
Image loader counter for: 8 name: classic-car :  1
Image loader Final counter for: 9 name: classic-car :  1
Image lo